In [1]:
import difflib
import numpy as np
import pandas as pd
from Levenshtein import distance

In [2]:
data = pd.read_csv('../input/geo1.csv', error_bad_lines=False, sep=';')
data.isnull().values.any()

True

In [3]:
data.dropna(inplace=True)
data.reset_index(drop=True, inplace=True)

**Расстояние Левенштейна**

In [4]:
def levenshtein_distance(df_dirty, df_clear):
    return np.sum([distance(str(dirty_add),
                            str(clear_add)) for dirty_add,
                                                clear_add in np.nditer([df_dirty, df_clear],
                                                                       flags=['refs_ok'])])

In [5]:
print(f'Расстояние Левенштейна между "грязными" и "чистыми" адресами: {levenshtein_distance(data.iloc[:, 0], data.iloc[:, 1])}')

Расстояние Левенштейна между "грязными" и "чистыми" адресами: 1162579


In [6]:
# function to alter string with mask that contains '?', '^' elements
def word(line, mask):
    caret = mask.rfind('^')
    whitespace = line.rfind(' ')
    if caret > whitespace:
        return line[whitespace + 1:].lower()
    else:
        return line[:caret + 1].lower()


# dataframe for changes detection
changes = pd.DataFrame({'dirty': [], 'clear': []})

# compare addresses using difflib
for dirty_add, clear_add in np.nditer([data.iloc[:, 0], data.iloc[:, 1]], flags=['refs_ok']):
    modifications = []
    for line in difflib.Differ().compare(str(dirty_add).replace(', ',',').strip().split(','),
                                         str(clear_add).replace(', ',',').strip().split(',')):
        
        # save all lines that relate to changes
        if line.startswith(('+', '-', '?')):
            modifications.append(line.replace('\n','').strip())
    
    # processing of different types of comparisons from difflib
    if all(modification.startswith('- ') for modification in modifications):
        for modification in modifications:
            changes = changes.append(pd.DataFrame({'dirty': [modification.strip('- ')],
                                                   'clear': ['']}),
                                     ignore_index=True)
    else:
        n_cycles = 0
        if any(modification.startswith('? ') for modification in modifications):
            n_cycles = len(modifications) - 3
        else:
            n_cycles = len(modifications) - 1
        
        mod_index = 99
        for mod_i in range(n_cycles):
            following = modifications[mod_i + 1]
            current = modifications[mod_i]
            
            if following.startswith('-') and current.startswith('+'):
                for modification in modifications:
                    if modification.endswith(current.split(' ')[-1]) and \
                    modification.startswith('-'):
                        if current.split(' ')[-1] == modification.split(' ')[-1]:
                            changes = changes.append(pd.DataFrame({'dirty': [modification[2:modification.rfind(' ')].lower()],
                                                                   'clear': [current[2:current.rfind(' ')].lower()]}),
                                                     ignore_index=True)
                        else:
                            changes = changes.append(pd.DataFrame({'dirty': [modification[2:].lower()],
                                                                   'clear': [current[2:].lower()]}),
                                                     ignore_index=True)
                        mod_index = modifications.index(modification)
            
            if following.startswith(('-', '+')) and current.startswith('-') and mod_i != mod_index:
                    changes = changes.append(pd.DataFrame({'dirty': [current.strip('- ').strip('+ ')],
                                                           'clear': ['']}), ignore_index=True)

            if following.startswith('?') and current.startswith(('-')):
                dirty = current.strip('- ')
                clear = modifications[mod_i + 2].strip('+ ')
                changes = changes.append(pd.DataFrame({'dirty': [word(dirty,
                                                                      following[2:]).lower()],
                                                       'clear': [word(clear,
                                                                      modifications[mod_i + 3][2:]).lower()]}),
                                         ignore_index=True)
            
            
changes.shape

(62094, 2)

Группируем и считаем количество изменений

In [7]:
changes = changes.groupby(['dirty','clear']) \
                 .size() \
                 .reset_index() \
                 .rename(columns={0:'count'}) \
                 .sort_values(by=['count'])

In [8]:
changes.tail(10)

,dirty,clear,count
1052,Ставропольский край,,996
557,Кировская область,,1069
759,Нижегородская область,,1104
908,Приморский край,,1133
633,Курганская область,,1535
827,Оренбургская область,,1616
986,Свердловская область,,1644
967,Россия,,1875
1531,село,с.,2000
950,Республика Башкортостан,,2768


Объединяем изменения с одинаковыми ключевыми словами

In [10]:
mutations = changes.copy()
keywords = ['область', 'обл.', 'обл', 'край', 'республика', 'респ', 'автономный округ', 'район' , 'р-н', 'городской округ']
for keyword in keywords:
    to_drop = mutations.where(((mutations['dirty'].str.contains(r'(?i)(\b){}($|,| )'.format(keyword))) | \
                               (mutations['dirty'].str.contains(r'(?i)(\b){}($|,| )'.format(keyword)))) & \
                               (mutations['clear'] == '')).dropna()
    mutations = mutations.drop(to_drop.index).append(pd.DataFrame({'dirty' : [keyword],
                                                                   'clear' : [''],
                                                                   'count': [to_drop['count'].sum()]}),
                                                     ignore_index=True)


mutations = mutations.sort_values(by=['count'], ascending=False).reset_index(drop=True)
mutations.head(10)

,dirty,clear,count
0,район,,22826.0
1,область,,15751.0
2,республика,,7963.0
3,край,,2937.0
4,р-н,,2453.0
5,село,с.,2000.0
6,Россия,,1875.0
7,обл.,,1712.0
8,улица,ул.,827.0
9,городской округ,,704.0


Применяем изменения и считаем расстояние Левенштейна. Если оно уменьшилось, отменяем изменение.

In [40]:
ld = levenshtein_distance(data.iloc[:, 0], data.iloc[:, 1])

for mutation in mutations.iloc[:, 0]:
    modifications = changes['dirty'].str.contains(r'(?i)(\b){}($|,| )'.format(mutation))
    to_change_values = pd.Series(changes.where(modifications)['clear'].values,
                                 index=changes.where(modifications)['dirty']).dropna().to_dict()

    mask = r'(?i)(\b)' + r'($|,| )|(\b)'.join(to_change_values.keys()) + r'($|,| )'
    to_change = data.iloc[:, 0].str.contains(mask)
    data_copy = data.copy()
    
    data_copy.loc[data_copy.iloc[:, 0] == \
                  data_copy.where(to_change).iloc[:, 0]] = data_copy.loc[data_copy.iloc[:, 0] == \
                                                                         data_copy.where(to_change).iloc[:, 0]] \
                                                                     .replace(to_change_values, regex=True)
    data_copy.iloc[:, 0] = data_copy.iloc[:, 0].str.strip(' ,').str.replace(' ,', '')
    
    print(levenshtein_distance(data_copy.iloc[:, 0], data.iloc[:, 1]))  

    if levenshtein_distance(data_copy.iloc[:, 0], data.iloc[:, 1]) < ld:
        data.loc[data.iloc[:, 0] == data.where(to_change).iloc[:, 0]] = data_copy.copy()
        ld = levenshtein_distance(data.iloc[:, 0], data.iloc[:, 1])
        
data

KeyboardInterrupt: 